# Notes

It is hard to evaluate if the georeferenced images make sense. They have been processed with the same time offset (210 seconds) as Måholmen. The starting and stopping times of the videos seem to coincide with GPS tracks that "look transect-like". However, the accuracy both in terms of position and in term og time is not very high. The transects should maybe be used as a more general tool for getting an overview and "feel" for the area, rather than a tool for exact georeferencing of specific features or areas. 

The Otter data is better suited for pinpointing specific places (but is of course limited by poor visibility in deep areas).


In [1]:
import video_transect, misc
from pathlib import Path
import pandas as pd

In [2]:
# Paths
# transect_dirs = [Path('/media/mha114/Massimal2/Massimal/Smola_Skalmen/DropCamTransects/20230621/T1'),
#                  Path('/media/mha114/Massimal2/Massimal/Smola_Skalmen/DropCamTransects/20230621/T2'),
#                  Path('/media/mha114/Massimal2/Massimal/Smola_Skalmen/DropCamTransects/20230621/T3'),
#                  Path('/media/mha114/Massimal2/Massimal/Smola_Skalmen/DropCamTransects/20230621/T4'),
#                  Path('/media/mha114/Massimal2/Massimal/Smola_Skalmen/DropCamTransects/20230621/T5')]

# transect_dirs = [Path('/media/mha114/Massimal2/Massimal/Smola_Skalmen/DropCamTransects/20230621/T1'),
#                  Path('/media/mha114/Massimal2/Massimal/Smola_Skalmen/DropCamTransects/20230621/T5')]

transect_dirs = [Path('/media/mha114/Massimal2/Massimal/Smola_Skalmen/DropCamTransects/20230621/T2'),
                 Path('/media/mha114/Massimal2/Massimal/Smola_Skalmen/DropCamTransects/20230621/T3'),
                 Path('/media/mha114/Massimal2/Massimal/Smola_Skalmen/DropCamTransects/20230621/T4')]

csv_file  = Path('/media/mha114/Massimal2/Massimal/Smola_Skalmen/DropCamTransects/20230621/GPS_Tracks/skippo_tracks/20230621_skippo_smola_skalmen.csv')

In [3]:
# Parameters
# sample_distance = 1.0 # Units defined by CRS, default is UTM with unit meters
sample_distance = 5 # Units defined by CRS, default is UTM with unit meters
video_offset_sec = 210 #235 #223 #135 # Time offset between Skippo and GoPro

# Note
Skippo CSV file does not seem to include the first transect. Therefore, only transects 2-4 are included here.

In [4]:
# Read "Skippo" CSV file
skippo_gdf = video_transect.yx_csv_to_geodataframe(csv_file,time_column_name='DateTimeS')
skippo_gdf = skippo_gdf.dropna() # Drop points without timestamps (at end of file)
skippo_gdf = skippo_gdf.sort_values('Time')
skippo_gdf = video_transect.filter_gdf_on_distance(skippo_gdf,sample_distance)
skippo_gdf

,Lat,Lng,Time,geometry
2552,63.459862,7.76221,2023-06-18 15:35:36+00:00,POINT (7.76221 63.45986)
2553,63.459911,7.76216,2023-06-18 15:35:37+00:00,POINT (7.76216 63.45991)
2554,63.459961,7.76208,2023-06-18 15:35:38+00:00,POINT (7.76208 63.45996)
2556,63.460060,7.76197,2023-06-18 15:35:40+00:00,POINT (7.76197 63.46006)
2559,63.460209,7.76179,2023-06-18 15:35:43+00:00,POINT (7.76179 63.46021)
...,...,...,...,...
2001,63.466282,7.77047,2023-06-21 17:10:12+00:00,POINT (7.77047 63.46628)
2009,63.466251,7.77056,2023-06-21 17:10:28+00:00,POINT (7.77056 63.46625)
2013,63.466221,7.77064,2023-06-21 17:10:36+00:00,POINT (7.77064 63.46622)
2016,63.466179,7.77068,2023-06-21 17:10:42+00:00,POINT (7.77068 63.46618)


In [5]:
# NOTE: Do a "dry run" without extracting images and check that time offsets are correct

for transect_dir in transect_dirs:
    video_dir = transect_dir / 'Video'
    
    image_dir = transect_dir / 'ExtractedImages/Original'
    image_dir.mkdir(parents=True, exist_ok=True)
    print(f'{image_dir} created (if not created before)')

    gpkg_dir = transect_dir / 'ImagesGeopackage'
    gpkg_dir.mkdir(exist_ok=True)
    print(f'{gpkg_dir} created (if not created before)')

    gpkg_file = gpkg_dir / (transect_dir.stem + '_GeotaggedImages.gpkg')
    print(f'GPKG file name for saving results: {gpkg_file}')

    video_files = misc.file_pattern_search(str(video_dir), '*.[Mm][Pp]4')
    print(f'Found video files {[Path(vfile).name for vfile in video_files]}')
    video_data = video_transect.get_video_data(str(video_dir),tz='utc',video_time_offset=pd.Timedelta(hours=-2))
    display(video_data)
    
    gdf = video_transect.prepare_gdf_with_video_data(skippo_gdf,video_data,video_offset_sec=video_offset_sec)
    display(gdf)

    gdf = video_transect.extract_images_from_video(gdf,str(image_dir))
    gdf.to_file(gpkg_file, driver="GPKG")
    

/media/mha114/Massimal2/Massimal/Smola_Skalmen/DropCamTransects/20230621/T2/ExtractedImages/Original created (if not created before)
/media/mha114/Massimal2/Massimal/Smola_Skalmen/DropCamTransects/20230621/T2/ImagesGeopackage created (if not created before)
GPKG file name for saving results: /media/mha114/Massimal2/Massimal/Smola_Skalmen/DropCamTransects/20230621/T2/ImagesGeopackage/T2_GeotaggedImages.gpkg
Found video files ['GH010776.MP4', 'GH020776.MP4']


,FileName,CreationTime,DurationSec,StartTimeSec,StopTimeSec
0,/media/mha114/Massimal2/Massimal/Smola_Skalmen...,2023-06-21 14:59:14+00:00,531.531000,0.000,531.531000
1,/media/mha114/Massimal2/Massimal/Smola_Skalmen...,2023-06-21 14:59:14+00:00,470.536733,531.531,1002.067733


,Lat,Lng,Time,VideoFile,TimeRelToVideoStart,TimeRelToFileStartSec,geometry
1025,63.462841,7.76889,2023-06-21 14:55:45+00:00,/media/mha114/Massimal2/Massimal/Smola_Skalmen...,0 days 00:00:01,1.0,POINT (7.76889 63.46284)
1029,63.462860,7.76899,2023-06-21 14:55:53+00:00,/media/mha114/Massimal2/Massimal/Smola_Skalmen...,0 days 00:00:09,9.0,POINT (7.76899 63.46286)
1034,63.462879,7.76910,2023-06-21 14:56:03+00:00,/media/mha114/Massimal2/Massimal/Smola_Skalmen...,0 days 00:00:19,19.0,POINT (7.76910 63.46288)
1039,63.462898,7.76921,2023-06-21 14:56:13+00:00,/media/mha114/Massimal2/Massimal/Smola_Skalmen...,0 days 00:00:29,29.0,POINT (7.76921 63.46290)
1044,63.462921,7.76931,2023-06-21 14:56:23+00:00,/media/mha114/Massimal2/Massimal/Smola_Skalmen...,0 days 00:00:39,39.0,POINT (7.76931 63.46292)
1048,63.462952,7.76939,2023-06-21 14:56:31+00:00,/media/mha114/Massimal2/Massimal/Smola_Skalmen...,0 days 00:00:47,47.0,POINT (7.76939 63.46295)
1053,63.462978,7.76948,2023-06-21 14:56:41+00:00,/media/mha114/Massimal2/Massimal/Smola_Skalmen...,0 days 00:00:57,57.0,POINT (7.76948 63.46298)
1056,63.463020,7.76955,2023-06-21 14:56:47+00:00,/media/mha114/Massimal2/Massimal/Smola_Skalmen...,0 days 00:01:03,63.0,POINT (7.76955 63.46302)
1062,63.463058,7.76964,2023-06-21 14:56:59+00:00,/media/mha114/Massimal2/Massimal/Smola_Skalmen...,0 days 00:01:15,75.0,POINT (7.76964 63.46306)
1066,63.463100,7.76970,2023-06-21 14:57:07+00:00,/media/mha114/Massimal2/Massimal/Smola_Skalmen...,0 days 00:01:23,83.0,POINT (7.76970 63.46310)


100%|██████████| 29/29 [00:22<00:00,  1.31it/s]


/media/mha114/Massimal2/Massimal/Smola_Skalmen/DropCamTransects/20230621/T3/ExtractedImages/Original created (if not created before)
/media/mha114/Massimal2/Massimal/Smola_Skalmen/DropCamTransects/20230621/T3/ImagesGeopackage created (if not created before)
GPKG file name for saving results: /media/mha114/Massimal2/Massimal/Smola_Skalmen/DropCamTransects/20230621/T3/ImagesGeopackage/T3_GeotaggedImages.gpkg
Found video files ['GX018277.MP4', 'GX028277.MP4']


...

100%|██████████| 111/111 [01:19<00:00,  1.39it/s]


/media/mha114/Massimal2/Massimal/Smola_Skalmen/DropCamTransects/20230621/T4/ExtractedImages/Original created (if not created before)
/media/mha114/Massimal2/Massimal/Smola_Skalmen/DropCamTransects/20230621/T4/ImagesGeopackage created (if not created before)
GPKG file name for saving results: /media/mha114/Massimal2/Massimal/Smola_Skalmen/DropCamTransects/20230621/T4/ImagesGeopackage/T4_GeotaggedImages.gpkg
Found video files ['GX018278.MP4', 'GX028278.MP4', 'GX038278.MP4']


,FileName,CreationTime,DurationSec,StartTimeSec,StopTimeSec
0,/media/mha114/Massimal2/Massimal/Smola_Skalmen...,2023-06-21 16:09:56+00:00,704.704000,0.000,704.704000
1,/media/mha114/Massimal2/Massimal/Smola_Skalmen...,2023-06-21 16:09:56+00:00,704.704000,704.704,1409.408000
2,/media/mha114/Massimal2/Massimal/Smola_Skalmen...,2023-06-21 16:09:56+00:00,272.038433,1409.408,1681.446433


,Lat,Lng,Time,VideoFile,TimeRelToVideoStart,TimeRelToFileStartSec,geometry
4629,63.463589,7.77600,2023-06-21 16:10:36+00:00,/media/mha114/Massimal2/Massimal/Smola_Skalmen...,0 days 00:04:10,250.000,POINT (7.77600 63.46359)
4633,63.463539,7.77598,2023-06-21 16:10:52+00:00,/media/mha114/Massimal2/Massimal/Smola_Skalmen...,0 days 00:04:26,266.000,POINT (7.77598 63.46354)
4637,63.463490,7.77593,2023-06-21 16:11:08+00:00,/media/mha114/Massimal2/Massimal/Smola_Skalmen...,0 days 00:04:42,282.000,POINT (7.77593 63.46349)
4642,63.463440,7.77591,2023-06-21 16:11:28+00:00,/media/mha114/Massimal2/Massimal/Smola_Skalmen...,0 days 00:05:02,302.000,POINT (7.77591 63.46344)
4645,63.463390,7.77590,2023-06-21 16:11:40+00:00,/media/mha114/Massimal2/Massimal/Smola_Skalmen...,0 days 00:05:14,314.000,POINT (7.77590 63.46339)
...,...,...,...,...,...,...,...
4979,63.463371,7.77756,2023-06-21 16:33:56+00:00,/media/mha114/Massimal2/Massimal/Smola_Skalmen...,0 days 00:27:30,240.592,POINT (7.77756 63.46337)
4982,63.463421,7.77719,2023-06-21 16:34:08+00:00,/media/mha114/Massimal2/Massimal/Smola_Skalmen...,0 days 00:27:42,252.592,POINT (7.77719 63.46342)
4984,63.463459,7.77686,2023-06-21 16:34:16+00:00,/media/mha114/Massimal2/Massimal/Smola_Skalmen...,0 days 00:27:50,260.592,POINT (7.77686 63.46346)
4985,63.463440,7.77661,2023-06-21 16:34:20+00:00,/media/mha114/Massimal2/Massimal/Smola_Skalmen...,0 days 00:27:54,264.592,POINT (7.77661 63.46344)


100%|██████████| 118/118 [01:29<00:00,  1.32it/s]
